<a href="https://colab.research.google.com/github/CameronKenworthyCode/python/blob/main/missing_wtmf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#initialize notebook
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re

!pip install pykml
from pykml.factory import KML_ElementMaker as KML

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

drive.mount('/content/drive/', force_remount=True)

file_list = gdrive.ListFile(
    {'q': "'1MjZTaxYDQ4JU-7TbBI0FwYDPHnrMbLIY' in parents and trashed=False"}).GetList()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 KB 1.1 MB/s eta 0:00:00
Mounted at /content/drive/


In [ ]:
def wtmf_parse(id):
  

In [ ]:
wtmf = pd.read_excel('/content/drive/MyDrive/WTMF/WTMF-Export-02-07-2023.xlsx')

ne_wtfm = []

for i in range(len(wtmf)):
  row = wtmf.iloc[i]
  if row['REF ID'].count("NE") > 0:
    ne_wtfm.append(i)

wtmf = wtmf.iloc[ne_wtfm]

wtmf_id = {}
for i in range(len(wtmf)):
  id = wtmf.iloc[i]['REF ID']
  id_list = id.split('-')
  wtmf_id[id] = {'region': id_list[0], 'wira': id_list[1], 'year': id_list[2], 'serial': id_list[3]}

wtmf_id = pd.DataFrame(wtmf_id).T
wtmf_id['year'] = wtmf_id['year'].astype(int)
wtmf_id['wira'] = wtmf_id['wira'].astype(int)
wtmf_id['serial'] = wtmf_id['serial'].astype(int)
wtmf_id = wtmf_id.sort_values(by=['year', 'serial'])

wtmf_id

,region,wira,year,serial
NE-58-14-0001,NE,58,14,1
NE-49-14-0002,NE,49,14,2
NE-55-14-0003,NE,55,14,3
NE-54-14-0004,NE,54,14,4
NE-56-14-0005,NE,56,14,5
...,...,...,...,...
NE-61-22-0081,NE,61,22,81
NE-54-22-0082,NE,54,22,82
NE-59-22-0083,NE,59,22,83
NE-49-22-0084,NE,49,22,84


In [ ]:
def find_missing(lst):
    return sorted(set(range(lst[0], lst[-1])) - set(lst))

def get_serial_range(df):
  s_range = []
  for i in range(len(df)):
    row = df.iloc[i]
    s_range.append(row['serial'])
  
  return s_range

def generate_ids(id_list, year):
  ids = []
  for i in range(len(id_list)):
    ids.append('NE-XX-' + str(year) + '-' + str(id_list[i]))
  return ids

missing = []

for year in wtmf_id['year'].unique():
  missing.append(generate_ids(find_missing(get_serial_range(wtmf_id.query('year == @year'))), year))

for year in missing:
  if len(year) > 0:
    print(year)

['NE-XX-14-52', 'NE-XX-14-60', 'NE-XX-14-65', 'NE-XX-14-101', 'NE-XX-14-102', 'NE-XX-14-132']
['NE-XX-16-78', 'NE-XX-16-87']
['NE-XX-17-6', 'NE-XX-17-18', 'NE-XX-17-45', 'NE-XX-17-74', 'NE-XX-17-75']
['NE-XX-18-99', 'NE-XX-18-116']
['NE-XX-19-33', 'NE-XX-19-69']
['NE-XX-20-39', 'NE-XX-20-42', 'NE-XX-20-43']
['NE-XX-21-35']
